# Satellite Imagery-Based Property Valuation
## Preprocessing and Exploratory Data Analysis

This notebook covers:
1. Data Loading and Exploration
2. Data Cleaning
3. Feature Engineering
4. Exploratory Data Analysis (EDA)
5. Geospatial Analysis
6. Satellite Image Visualization

In [ ]:
# Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from PIL import Image
import warnings

warnings.filterwarnings('ignore')
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("✓ Libraries imported!")


## 1. Data Loading

In [ ]:
# Load Data
train_df = pd.read_excel('data/raw/train.xlsx')
test_df = pd.read_excel('data/raw/test.xlsx')

print(f"Training data: {train_df.shape}")
print(f"Test data: {test_df.shape}")
print(f"\nColumns: {list(train_df.columns)}")
train_df.head()


In [ ]:
# Data Info
print("=== Data Types ===")
print(train_df.dtypes)
print("\n=== Missing Values ===")
print(train_df.isnull().sum())
print("\n=== Statistics ===")
train_df.describe()


## 2. Feature Engineering

In [ ]:
def engineer_features(df):
    """Create new features from existing data."""
    df = df.copy()
    
    # Renovation features
    df['is_renovated'] = (df['yr_renovated'] > 0).astype(int)
    df['yr_renovated_filled'] = df['yr_renovated'].copy()
    df.loc[df['yr_renovated'] == 0, 'yr_renovated_filled'] = df.loc[df['yr_renovated'] == 0, 'yr_built']
    
    # Age features
    df['house_age'] = 2025 - df['yr_built']
    df['years_since_renovation'] = 2025 - df['yr_renovated_filled']
    
    # Room features
    df['total_rooms'] = df['bedrooms'] + df['bathrooms']
    df['bath_bed_ratio'] = df['bathrooms'] / (df['bedrooms'] + 1)
    
    # Space features
    df['living_lot_ratio'] = df['sqft_living'] / df['sqft_lot']
    df['above_living_ratio'] = df['sqft_above'] / df['sqft_living']
    df['has_basement'] = (df['sqft_basement'] > 0).astype(int)
    
    # Neighborhood features
    df['living_vs_neighbors'] = df['sqft_living'] / (df['sqft_living15'] + 1)
    df['lot_vs_neighbors'] = df['sqft_lot'] / (df['sqft_lot15'] + 1)
    
    # Quality features
    df['quality_score'] = df['grade'] * df['condition']
    df['is_luxury'] = ((df['grade'] >= 11) | (df['waterfront'] == 1) | (df['view'] >= 3)).astype(int)
    
    return df

# Apply feature engineering
train_df = engineer_features(train_df)
test_df = engineer_features(test_df)

print(f"✓ Feature engineering complete!")
print(f"Training: {train_df.shape}")
print(f"New features: {train_df.shape[1] - 21}")


## 3. Exploratory Data Analysis

In [ ]:
# Price Distribution
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Original price
axes[0].hist(train_df['price'], bins=50, edgecolor='black', alpha=0.7)
axes[0].set_title('Price Distribution', fontweight='bold')
axes[0].set_xlabel('Price ($)')
axes[0].axvline(train_df['price'].median(), color='red', linestyle='--', 
                label=f'Median: ${train_df["price"].median():,.0f}')
axes[0].legend()

# Log-transformed price
axes[1].hist(np.log1p(train_df['price']), bins=50, edgecolor='black', alpha=0.7, color='green')
axes[1].set_title('Log-Transformed Price', fontweight='bold')
axes[1].set_xlabel('Log(Price)')

plt.tight_layout()
plt.show()

print(f"Price range: ${train_df['price'].min():,.0f} - ${train_df['price'].max():,.0f}")
print(f"Median: ${train_df['price'].median():,.0f}")


In [ ]:
# Correlation Analysis
numerical_cols = train_df.select_dtypes(include=[np.number]).columns.tolist()
numerical_cols = [col for col in numerical_cols if col not in ['id', 'date']]

correlations = train_df[numerical_cols].corr()['price'].sort_values(ascending=False)
print("Top 10 features correlated with price:")
print(correlations.head(10))


In [ ]:
# Correlation Heatmap
top_features = correlations.head(10).index.tolist()
plt.figure(figsize=(10, 8))
sns.heatmap(train_df[top_features].corr(), annot=True, fmt='.2f', cmap='coolwarm', center=0)
plt.title('Correlation Heatmap - Top Features', fontweight='bold')
plt.tight_layout()
plt.show()


In [ ]:
# Price by Key Features
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
axes = axes.ravel()

features = ['waterfront', 'view', 'grade', 'condition']
for idx, feat in enumerate(features):
    train_df.groupby(feat)['price'].mean().plot(kind='bar', ax=axes[idx], color='steelblue')
    axes[idx].set_title(f'Average Price by {feat}', fontweight='bold')
    axes[idx].set_ylabel('Price ($)')
    axes[idx].tick_params(axis='x', rotation=0)

plt.tight_layout()
plt.show()


## 4. Geospatial Analysis

In [ ]:
# Geographic Price Distribution
plt.figure(figsize=(12, 8))
scatter = plt.scatter(train_df['long'], train_df['lat'], 
                      c=train_df['price'], cmap='YlOrRd', alpha=0.5, s=20)
plt.colorbar(scatter, label='Price ($)')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('Property Prices by Location', fontweight='bold')
plt.tight_layout()
plt.show()

# Waterfront premium
waterfront_avg = train_df.groupby('waterfront')['price'].mean()
premium = (waterfront_avg[1] / waterfront_avg[0] - 1) * 100
print(f"\nWaterfront premium: {premium:.1f}%")


## 5. Satellite Image Samples

In [ ]:
# Display sample satellite images at different price points
images_dir = Path('data/images/train')
images = list(images_dir.glob('*.png'))

if len(images) > 0:
    print(f"Found {len(images)} training images")
    
    # Get samples at different price points
    price_quantiles = train_df['price'].quantile([0.1, 0.3, 0.5, 0.7, 0.9]).values
    
    fig, axes = plt.subplots(1, 5, figsize=(20, 4))
    for idx, q_price in enumerate(price_quantiles):
        sample_idx = (train_df['price'] - q_price).abs().idxmin()
        prop_id = train_df.loc[sample_idx, 'id']
        price = train_df.loc[sample_idx, 'price']
        
        img_path = images_dir / f"{prop_id}.png"
        if img_path.exists():
            img = Image.open(img_path)
            axes[idx].imshow(img)
            axes[idx].set_title(f'${price:,.0f}', fontweight='bold')
            axes[idx].axis('off')
    
    plt.suptitle('Satellite Images at Different Price Points', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()
else:
    print("No images found. Run data_fetcher.py first.")


## 6. Save Processed Data

In [ ]:
# Log-transform price for stable training
train_df['price_original'] = train_df['price']
train_df['price'] = np.log1p(train_df['price'])

print("✓ Price normalized with log transform")
print(f"Original: ${train_df['price_original'].min():,.0f} - ${train_df['price_original'].max():,.0f}")
print(f"Log: {train_df['price'].min():.2f} - {train_df['price'].max():.2f}")

# Save processed data
Path('data/processed').mkdir(parents=True, exist_ok=True)
train_df.to_csv('data/processed/train_processed.csv', index=False)
test_df.to_csv('data/processed/test_processed.csv', index=False)

print("\n✓ Processed data saved!")
print(f"Training: data/processed/train_processed.csv ({len(train_df)} samples)")
print(f"Test: data/processed/test_processed.csv ({len(test_df)} samples)")
